In [1]:
from selenium.webdriver import Chrome
from time import sleep
import json
import pandas as pd
from selenium.webdriver.common.by import By  # Import By class
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options





In [60]:

def getMatches(url):

    #Initial dataframe for all matches of that day
    df = pd.DataFrame(columns=['game', 'link'])

    # Set up Chrome options to run in headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")


    #Create browser connection
    browser = Chrome()
    #first_page = 'https://www.hltv.org/results'
    browser.get(url)
    sleep(2)


    # Find elements by class name 'a-reset'
    games = browser.find_elements(By.CLASS_NAME, 'a-reset')
    
    #Get the link and the matche name 
    for i in range(len(games)):
        df.loc[i, 'game'] = games[i].text
        df.loc[i, 'link'] = games[i].get_attribute('href')
    
    #Drop duplicates
    df.drop_duplicates(inplace=True)

    #Only get the matches 
    df2 = df[(df['game'] != 'BENCH') & (df['game'] != '') ]
    df2 = df2[df2['link'].str.contains('/matches/') & df2['link'].notna()]

    #Split the query to get the game, teams, score, event and type of match
    df2['split_game'] = df2['game'].str.split('\n')

    #Check if they are really matches (needs teams, score and type of match making 5 columns)
    df3 =  df2[df2['split_game'].apply(lambda x: len(x) > 2)]

    #Extract the matchid 
    df3['match_id'] = df3['link'].str.extract(r'/matches/(\d+)/')

    #Get the match details
    df3['team_1']= df3['split_game'].str[0]
    df3['team_2']=df3['split_game'].str[2]
    df3['result']=df3['split_game'].str[1]
    df3['event']=df3['split_game'].str[3]
    df3['match_type']=df3['split_game'].str[4]

    #get the result of each team
    df3['result']=df3['result'].str.split('-')
    df3['score_t1'] =df3['result'].str[0]
    df3['score_t2'] =df3['result'].str[1]

    # Change the datatype of score_1 and score_2 to int
    df3['score_t1'] = df3['score_t1'].astype(int)
    df3['score_t2'] = df3['score_t2'].astype(int)

    #Get the match winner
    df3['Winner'] = df3.apply(lambda row: row['team_1'] if row['score_t1'] > row['score_t2'] else row['team_2'], axis=1)

    df3.drop(columns=['game','split_game','result'],inplace=True)

    browser.quit()

    return df3
def getplayers(url):


    #Create browser connection
    browser = Chrome()
    browser.get(url)
    
    lineups = browser.find_element(By.CLASS_NAME, 'lineups').find_elements(By.CLASS_NAME, 'players')
    p = []
    for lineup in lineups:
        players=lineup.find_elements(By.CLASS_NAME, 'player')
        for player in players:
            href = player.find_element(By.TAG_NAME, 'a').get_attribute('href')
            p.append(href)
             
    df1 = pd.DataFrame(p, columns=['player']).drop_duplicates()

    browser.quit()

    return df1


def playerdetails(url,df):
    # Set up Chrome options to run in headless mode
    #chrome_options = Options()
    #chrome_options.add_argument("--headless")


    #Create browser connection
    browser = Chrome()
    browser.get(url)       
    sleep(2)

    playerInfoWrapper = browser.find_element(By.CLASS_NAME, 'playerInfoWrapper')

    playerNameWrapper = playerInfoWrapper.find_element(By.CLASS_NAME, 'playerNameWrapper')
    playername = playerNameWrapper.find_element(By.CLASS_NAME, 'playerRealname') #name
    playerInfo = playerInfoWrapper.find_element(By.CLASS_NAME, 'playerInfo')
        
    df.loc[df['player'] == url, 'Name'] = playername.text
    df.loc[df['player'] == url,'nationality'] = playername.find_element(By.TAG_NAME, 'img').get_attribute('alt')
    info = playerInfo.text.split('\n')
    df.loc[df['player'] == url,'age'] = info[1]
    df.loc[df['player'] == url,'team'] = info[3]

    browser.quit()
    
def getevents(url):

    #Create browser connection
    browser = Chrome()
    browser.get('url')
        
    events = browser.find_element(By.CLASS_NAME, 'timeAndEvent').find_element(By.CLASS_NAME, 'text-ellipsis')

    id = events.find_element(By.TAG_NAME, 'a').get_attribute('href')
    name = events.text
             
    df1 = pd.DataFrame(pcolumns=['id','event'])
    df1['id'] = id
    df1['event']=name

    browser.quit()

    return df1


In [27]:
#gettin only the first page of results (could get more :/)
first_page = 'https://www.hltv.org/results'
matches =getMatches(first_page)

C:\Users\filip\AppData\Local\Temp\ipykernel_33912\27281376.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['match_id'] = df3['link'].str.extract(r'/matches/(\d+)/')
C:\Users\filip\AppData\Local\Temp\ipykernel_33912\27281376.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['team_1']= df3['split_game'].str[0]
C:\Users\filip\AppData\Local\Temp\ipykernel_33912\27281376.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [1]:
#Get all the players in each match
players = pd.DataFrame(columns=['player'])
for i in matches['link']:
    pd.concat([players,getplayers(matches.iloc[0,0]]))
players.drop_duplicates(inplace=True)
#Extract the matchid 
players['player_id'] = players['player'].str.extract(r'/player/(\d+)/')

NameError: name 'pd' is not defined

In [37]:
#get the details for each player
for i in players['player']:
    playerdetails(i,players)


,player
0,https://www.hltv.org/player/18462/mezii
1,https://www.hltv.org/player/7322/apex
2,https://www.hltv.org/player/18221/spinx
3,https://www.hltv.org/player/11893/zywoo
4,https://www.hltv.org/player/16693/flamez
10,https://www.hltv.org/player/2241/roman
11,https://www.hltv.org/player/20462/story
12,https://www.hltv.org/player/20464/ewjerkz
13,https://www.hltv.org/player/20465/arrozdoce
14,https://www.hltv.org/player/630/mutiris


In [42]:
#get events
events = pd.DataFrame(pcolumns=['id','event'])
for i in matches['link']:
    pd.concat(events,getevents(i))
events.drop_duplicates(inplace=True)
players['event_id'] = players['player'].str.extract(r'/events/(\d+)/')

,link,match_id,team_1,team_2,event,match_type,score_t1,score_t2,Winner
0,https://www.hltv.org/matches/2374325/vitality-...,2374325,Vitality,SAW,IEM Cologne 2024,bo3,2,0,Vitality
1,https://www.hltv.org/matches/2374322/mouz-vs-g...,2374322,MOUZ,G2,IEM Cologne 2024,bo3,2,0,MOUZ
2,https://www.hltv.org/matches/2374323/saw-vs-fa...,2374323,SAW,FaZe,IEM Cologne 2024,bo3,2,1,SAW
3,https://www.hltv.org/matches/2374749/ruby-vs-n...,2374749,RUBY,Nexus,CCT Season 2 Europe Series 9,bo3,2,1,RUBY
4,https://www.hltv.org/matches/2374832/b8-vs-laz...,2374832,B8,Lazer Cats,ESL Challenger Atlanta 2024 Europe Open Qualifier,mrg,19,17,B8
...,...,...,...,...,...,...,...,...,...
98,https://www.hltv.org/matches/2374641/fluffy-ai...,2374641,FLUFFY AIMERS,Limitless,ESL Challenger League Season 48 North America,mrg,13,5,FLUFFY AIMERS
99,https://www.hltv.org/matches/2374671/bestia-vs...,2374671,BESTIA,MIBR,ESL Challenger League Season 48 South America,mrg,16,14,BESTIA
100,https://www.hltv.org/matches/2374669/case-vs-o...,2374669,Case,ODDIK,ESL Challenger League Season 48 South America,anc,16,19,ODDIK
101,https://www.hltv.org/matches/2374670/solid-vs-...,2374670,Solid,Fluxo,ESL Challenger League Season 48 South America,vtg,13,8,Solid
